RAG의 기본 개념 / 문서 전처리 과정의 이해

학습 목표
- RAG(Retrieval-Augmented Generation)의 기본 개념 이해
- 문서 로딩, 청크 분할, 임베딩, 벡터 저장 과정 실습
- LangChain을 활용한 RAG 파이프 라인 구축
- 검색 기반 질의 응답 시스템 구현

1. RAG 개념 및 아키텍처
	- Rag란?
		- RAG(Rentrieval-Augmented Generation)는 기존 LLM의 한계를 보완하기 위한 방법론
			- 문제점 : LLM은 훈련 시점의 고정된 데이터에만 의존
			- 해결책 : 외부 지식 베이스를 동적으로 검색하여 응답 생성 시 활용
			- 장점 : 최신 정보, 도메인 특화 지식, 사실 기반 응답 가능

	- RAG의 핵심 구성요서
		1. 검색(Retrieval) 시스템
			- 임베딩 모델 : 텍스트를 벡터로 변환
			- 벡터 데이터베이스 : 임베딩 벡터 저장 및 인덱싱
			- 유사도 검색 : 코사인 유사도, 유클리 거리 등

		2. 증강(Augmentation)
			- 검색된 문서 전처리 및 포맷팅
			- 프롬프트 엔지니어링
			- 컨텍스트 길이 관리
		
		3. 생성(Generation)
			- LLM을 통한 최종 응답 생성
			- 검색된 컨텍스트와 원본 질의 결합

2. 최신 LangChain을 활용한 RAG구현
	- step1. Indexing
		1. 문서 수집 및 전처리
		2. 문서 청크 분할
		3. 임베딩 생성
		4. 벡터 저장소 구축

	- 문서 청크 분할(Split Texts)
		- 불러온 데이터를 작은 크기의 단위 chunk로 분할하는 과정
		- 자연어 처리(NLP) 기술을 활용하여 큰 문서를 처리가 쉽도록 문단, 문장 또는 구 단위로 나누는 작업
		- 검색 효율성을 높이기 위한 중요한 과정
			1. 청크 크기 선택
				- 너무 작은 청크 : 문맥 손실
				- 너무 큰 청크 : 관련성 저하
			2. 중복 영역 설정
				- 문맥 유지를 위해 필요
				- 일반적으로 10-20% 권장
		- LangChain기본 Text Splitter종류
			- `RecursiveCharacterTextSplitter`
				- 여러 구분자를 우선순위대로 시도하여 문서를 자연스럽게 분할
				- 기본 구분자 순서 : `["\n\n", "\n", " ", ""]`
				- 대부분의 경우에 가장 좋은 성능을 보임
			- `CharacterTextSplitter`
				- 단일 구분자로만 분할 (예: `\n\n`)
				- 간단한 경우에 사용
		- `chunk_size`
			- 엄격한 최대값이 아닌 목표값이기 때문에 `Created a chunk of size 9814, which is longer than the specified 1000`경고 나타날 수 있다.
			- `separator="\n\n"`로 문단 단위로 분할하기 때문에 하나의 문단이 `chunk_size`보다 크면 그대로 유지 된다.
			- 이는 문맥을 유지하기 위한 의도된 동작입니다.

3. 문서 임베딩 생성(Document Embeddings)
	- 임베딩 모델을 사용하여 텍스트를 벡터로 변환
	- 임베딩을 기반으로 유사성 검색에 사용
	- 임베딩 모델 선택
		- 성능가 비용 고려
		- 다국어 지원 여부 확인

4. 백터 저장소 구축 (Vectorstores)
	- 임베딩 벡터를 벡터저장소에 저장
	- 저장된 임베딩을 기반으로 유사성 검색을 수행하는데 활용

In [31]:
# 최신 LangChain을 활용한 RAG구현 - 1. 문서 수집 및 전처리
# 1. 문서 데이터 로드 (Load Data)
# 	- RAG에 사용할 데이터를 불러오는 단계 (검색에 사용될 지식이나 정보)
# 	- 외부 데이터 소스에서 정보를 수집하고, 필요한 형식으로 변환하여 시스템에 로드
import re
from langchain_community.document_loaders import WebBaseLoader # Data Loader - 웹페이지 데이터 가져오기
from langchain_core.documents import Document

# 1단계: 데이터 준비 - 웹문서 검색을 위해 관련 URL 가져오기
web_urls = [
    "https://n.news.naver.com/mnews/article/029/0002927209",
    "https://n.news.naver.com/mnews/article/092/0002358620",
    "https://n.news.naver.com/mnews/article/008/0005136824",
]


# 2단계: WebBaseLoader를 사용해 텍스트 로드
"""
힌트:
- WebBaseLoader를 사용하여 web_urls의 문서들을 로드하세요
- loader.load()를 호출하여 Document 객체 리스트를 얻습니다
- 로드된 문서의 개수를 출력하여 확인하세요

기대 출력:
- 로드된 문서 개수: 3개 (URL 개수와 동일)
"""
loader = WebBaseLoader(web_urls)

# 웹페이지 텍스트 > Document 객체로 변환
docs = loader.load()

# 라인 제거
def remove_boilerplate_lines(text: str) -> str:
    lines = []
    for line in text.splitlines():
        s = line.strip()
        if not s:
            continue

        # 기자 이메일 라인 제거 (예: 장유미 기자(sweet@zdnet.co.kr))
        if re.search(r"기자.*\([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\)", s):
            continue

        # 저작권/무단전재/재배포/AI 학습 금지 제거
        if any(k in s for k in ["Copyright", "All rights reserved", "무단 전재", "재배포", "AI 학습 이용 금지"]):
            continue

        # 네이버 공통 UI/메뉴/정책/구독 블록 제거(필요하면 계속 추가)
        if any(k in s for k in [
            "본문 바로가기", "구독", "댓글 정책", "랭킹", "많이 본 뉴스", "이슈 NOW", "기사 섹션"
        ]):
            continue

        lines.append(s)
    return "\n".join(lines)

# docs 정제하기
def clean_newlines(text: str) -> str:
    # 1) NBSP 같은 특수 공백 제거
    text = text.replace("\xa0", " ")

    # 2) 줄 끝 공백 제거
    text = re.sub(r"[ \t]+\n", "\n", text)

    # 3) 너무 많은 줄바꿈을 "문단 경계"로 통일
    #    - 핵심: separator="\n\n"가 의미를 가지게 만드는 것
    text = re.sub(r"\n{3,}", "\n\n", text)

    # 4) 공백 과다 압축
    text = re.sub(r"[ \t]{2,}", " ", text)
    return text.strip()

def trim_naver_article(text: str) -> str:
    end_markers = [
        "Copyright", "All rights reserved", "무단 전재", "언론사홈", "많이 본 뉴스", "댓글 정책", "랭킹 뉴스",
        "이슈 NOW", "함께 볼만한 뉴스", "기자 프로필", "기사 섹션 분류 안내"
    ]
    for marker in end_markers:
        idx = text.find(marker)
        if idx != -1 and idx > 0:
            text = text[:idx]
            break
    return text.strip()

clean_docs = [
    Document(
        page_content=remove_boilerplate_lines(
            trim_naver_article(
                clean_newlines(doc.page_content)
            )
        ),
        metadata=doc.metadata
    )
    for doc in docs
]

# 출력 : 로드된 문서 개수: 3개 (URL 개수와 동일)
# print(f"Document: {docs}")
# print(f"Document: {docs}")
# print(f"Document 개수: {len(docs)}")

for idx, doc in enumerate(clean_docs):
    print(f"========= {idx} ==========")
    print(f"title: {doc.metadata.get('title')}")
    print(f"len: {len(doc.page_content)}")
    print(doc.page_content[:400])

========= 0 ==========
title: "그래서 AGI 왔다고?"…오픈AI 올트먼, X에 오묘한 메시지
len: 1507
"그래서 AGI 왔다고?"…오픈AI 올트먼, X에 오묘한 메시지
NAVER
뉴스
엔터
스포츠
날씨
프리미엄
검색
언론사별
정치
경제
사회
생활/문화
IT/과학
세계
신문보기
오피니언
TV
팩트체크
알고리즘 안내
정정보도 모음
디지털타임스
디지털타임스
보러가기
닫기
닫기
PICK
안내
언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기
"그래서 AGI 왔다고?"…오픈AI 올트먼, X에 오묘한 메시지
입력
2025.01.05. 오후 5:48
수정
2025.01.05. 오후 5:50
기사원문
추천
반응
쏠쏠정보
0
흥미진진
0
공감백배
0
분석탁월
0
후속강추
0
댓글
반응
텍스트 음성 변환 서비스 사용하기
성별
남성
여성
말하기 속도
느림
보통
빠름
이동 통신망을 이용하여 음성을 재생하면 별도의 데이터
========= 1 ==========
title: AI에 진심인 MS, 1년간 데이터센터에 118兆 붓는다
len: 2623
AI에 진심인 MS, 1년간 데이터센터에 118兆 붓는다
NAVER
뉴스
엔터
스포츠
날씨
프리미엄
검색
언론사별
정치
경제
사회
생활/문화
IT/과학
세계
신문보기
오피니언
TV
팩트체크
알고리즘 안내
정정보도 모음
지디넷코리아
지디넷코리아
보러가기
닫기
닫기
PICK
안내
언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기
AI에 진심인 MS, 1년간 데이터센터에 118兆 붓는다
장유미 기자
입력
2025.01.05. 오후 2:31
기사원문
추천
반응
쏠쏠정보
0
흥미진진
0
공감백배
0
분석탁월
0
후속강추
0
댓글
반응
텍스트 음성 변환 서비스 사용하기
성별
남성
여성
말하기 속도
느림
보통
빠름
이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.
본문듣기 시
========= 2 ==========
title: "게임 만들 사람 줄어도 괜찮아" A

In [33]:
from langchain_text_splitters import CharacterTextSplitter # 문서 청크 분할(Split Texts)

# 3단계: CharacterTextSplitter로 문서 분할
"""
힌트:
- CharacterTextSplitter를 import하세요
- chunk_size=500, chunk_overlap=100으로 설정하세요
- separator="\n\n"로 문단 단위 분할
- split_documents() 메서드를 사용하여 문서를 분할하세요
- 분할된 청크의 개수를 출력하세요

기대 출력:
- 분할된 청크 개수: 약 20-40개 (문서 내용에 따라 다를 수 있음)
"""
text_splitter = CharacterTextSplitter(
	separator="\n\n",       # 문단 구분자 - 구분자를 기준으로 나눈다.
	chunk_size=500,         # 문단 길이
	chunk_overlap=100,      # 겹치는 길이
	length_function=len,    # 길이 측정 함수
	is_separator_regex=False,   # separator가 정규식인지 여부
)

splitted_docs = text_splitter.split_documents(clean_docs)

# 초반 청크 확인(필수)
for i, d in enumerate(splitted_docs[:10]):
    print(f"\n--- chunk {i} ---")
    print("source:", d.metadata.get("source"))
    print(d.page_content[:300])



--- chunk 0 ---
source: https://n.news.naver.com/mnews/article/029/0002927209
"그래서 AGI 왔다고?"…오픈AI 올트먼, X에 오묘한 메시지
NAVER
뉴스
엔터
스포츠
날씨
프리미엄
검색
언론사별
정치
경제
사회
생활/문화
IT/과학
세계
신문보기
오피니언
TV
팩트체크
알고리즘 안내
정정보도 모음
디지털타임스
디지털타임스
보러가기
닫기
닫기
PICK
안내
언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기
"그래서 AGI 왔다고?"…오픈AI 올트먼, X에 오묘한 메시지
입력
2025.01.05. 오후 5:48
수정
2025.01.05. 오후 5:50
기사원문
추천
반응
쏠쏠정보
0
흥미진진
0


--- chunk 1 ---
source: https://n.news.naver.com/mnews/article/092/0002358620
AI에 진심인 MS, 1년간 데이터센터에 118兆 붓는다
NAVER
뉴스
엔터
스포츠
날씨
프리미엄
검색
언론사별
정치
경제
사회
생활/문화
IT/과학
세계
신문보기
오피니언
TV
팩트체크
알고리즘 안내
정정보도 모음
지디넷코리아
지디넷코리아
보러가기
닫기
닫기
PICK
안내
언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기
AI에 진심인 MS, 1년간 데이터센터에 118兆 붓는다
장유미 기자
입력
2025.01.05. 오후 2:31
기사원문
추천
반응
쏠쏠정보
0
흥미진진
0
공감백배
0
분석탁월
0
후속강추
0
댓글


--- chunk 2 ---
source: https://n.news.naver.com/mnews/article/008/0005136824
"게임 만들 사람 줄어도 괜찮아" AI가 책임진다
NAVER
뉴스
엔터
스포츠
날씨
프리미엄
검색
언론사별
정치
경제
사회
생활/문화
IT/과학
세계
신문보기
오피니언
TV
팩트체크
알고리즘 안내
정정보도 모음
머니투데이
머니투데이
보러가기
닫기
닫기
"게임 만들 사람 줄어도 괜찮아" A

In [ ]:
# 글자 수 기준으로 엄격하게 분할하기
from langchain_text_splitters import CharacterTextSplitter

# 1000자씩 잘라서 Document로 전환
text_splitter = CharacterTextSplitter(
	separator="",
	chunk_size=1000,
	length_function=len,
	is_separator_regex=False,
)

equally_splitted_docs = text_splitter.split_documents(docs)

print(f"Document 개수 : {len(equally_splitted_docs)}")

Document 개수 : 43


In [ ]:
# 4단계: 임베딩 및 벡터 저장소 구현
"""
힌트:
- OpenAIEmbeddings를 사용하여 임베딩 모델을 생성하세요
- Chroma 벡터 저장소를 생성하세요
- add_documents()를 사용하여 분할된 문서를 벡터 저장소에 추가하세요
- 저장된 문서 개수를 확인하세요

기대 출력:
- 임베딩 모델 생성 완료
- 벡터 저장소에 저장된 문서 개수: (3단계의 청크 개수와 동일)
"""
import time # 컬렉션 이름을 매번 새로 만들기
from langchain_openai import OpenAIEmbeddings # OpenAI Embeddings - 문장 임베딩
from langchain_chroma import Chroma # Chroma 백터 저장소에 문서 저장하기


# OpenAIEmbeddings를 사용하여 embedding model 생성
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small") # 사용할 모델 이름을 지정 가능
# Chroma 벡터 저장소를 생성
# time으로 매번 이름을 매번 새롭게 만들기 - 재실행 누적문제로 retriver가 이상해지는걸 크게 줄일 수 있다.
vector_store = Chroma(
	collection_name=f"naver_news_{int(time.time())}", 
	embedding_function=embedding_model
)
# add_documents()를 사용하여 분할된 문서를 벡터 저장소에 추가
document_ids = vector_store.add_documents(splitted_docs)

# 저장된 문서 개수 확인
print(f"저장된 Document 개수 : {len(document_ids)} / 청크 개수: {len(splitted_docs)}")

## chroma 실행 시 Failed to send telemetry event 경고는 Chroma의 사용 통계 수집 기능과 관련된 것으로, 실제 기능에는 영향 주지 않는다.

저장된 Document 개수 : 15 / 청크 개수: 15


In [34]:
# 5단계: RAG 기반 QA 체인 구현
"""
힌트:
- ChatOpenAI로 LLM 모델을 생성하세요 (model="gpt-4.1-nano")
- ChatPromptTemplate으로 프롬프트 템플릿을 만드세요
- 벡터 저장소에서 as_retriever()로 검색기를 생성하세요
- RunnableParallel 사용하여 RAG 체인을 구성하세요

기대 출력:
- RAG 체인 생성 완료
"""
# RunableParallel을 사용한 RAG 체인
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# ChatOpenAI로 LLM 모델을 생성하세요 (model="gpt-4.1-nano") 
llm = ChatOpenAI(model="gpt-4.1-nano",temperature=0)
# llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)
# llm = ChatOpenAI(model="gpt-4.1",temperature=0)

# ChatPromptTemplate으로 프롬프트 템플릿 생성
# 시스템 프롬프트
system_prompt = (
	"너는 뉴스 요약 비서야, 제공된 자료에서만 근거를 사용해 답해. "
	"자료에 없는 내용은 '확인불가'라고 말해. 추측 금지. "
	"한국어로 간결하고 정확하게 답변해."
)

prompt = ChatPromptTemplate.from_messages([
	("system", system_prompt),
	("human", 
	"아래 자료를 바탕으로 질문에 답해줘.\n\n"
	"## 자료\n{context}\n\n"
	"## 질문\n{question}\n\n"
	"출력형식 : \n"
	"1) 요약: (3~5문장)\n"
	"2) 주요 인물: (이름 목록)\n"
	"3) 사건/이슈: (한 문장)\n"
	)
])

# 벡터 저장소에서 as_retriever()로 검색기를 생성
retriever = vector_store.as_retriever(
	search_type="mmr", # 다양성 확보 - 3개 기사에서 골고루 뽑히기 쉬움
	search_kwargs={"k": 12, "fetch_k": 30} # k를 늘려야 요약 자료가 생김
)

rag_chain = (
	RunnableParallel({
		"docs": retriever,
		"question": RunnablePassthrough()
	}) 
	| RunnableLambda(lambda x: {
		"context": "\n\n".join(doc.page_content for doc in x["docs"]), # docs -> context 문자열화
		"question": x["question"]
	})
	| prompt 
	| llm 
	| StrOutputParser()
)

# 6단계: QA 체인으로 질문 응답
"""
힌트:
- 뉴스 기사와 관련된 질문을 준비하세요
- rag_chain.invoke({"question": 질문})으로 답변을 얻으세요
- response['answer']로 답변 내용을 확인하세요
- response['context']로 검색된 문서를 확인하세요

기대 출력:
- 질문에 대한 답변이 출력됩니다
- 검색된 관련 문서들의 내용이 포함됩니다

예시 질문:
- "기사의 주요 내용을 요약해주세요"
- "주요 인물은 누구인가요?"
- "어떤 사건에 대한 기사인가요?"
"""

# query = "1. 기사의 내용을 요약해주세요. 2. 주요 인물은 누구인가요? 3. 어떤 사건에 대한 기사인가요?"
# response = rag_chain.invoke(query)

# print(f"\n답변:\n{response}")

hits = retriever.invoke("기사 내용을 요약해줘")
for idx, doc in enumerate(hits[:10]):
	print(f"\n--- hit {idx} ---") 
	print(f"source: {doc.metadata.get('source')}") 
	print(doc.page_content[:250])

hits = retriever.invoke("MS가 데이터센터에 800억 달러 투자한다는 내용 요약해줘")
for i, d in enumerate(hits[:5]):
    print(i, d.metadata.get("source"))
    print(d.page_content[:250])



--- hit 0 ---
source: https://n.news.naver.com/mnews/article/092/0002358620
장유미 기자(sweet@zdnet.co.kr)

--- hit 1 ---
source: https://n.news.naver.com/mnews/article/029/0002927209
"그래서 AGI 왔다고?"…오픈AI 올트먼, X에 오묘한 메시지

본문 바로가기

NAVER

뉴스

엔터

스포츠

날씨

프리미엄

검색

언론사별

정치

경제

사회

생활/문화

IT/과학

세계

랭킹

신문보기

오피니언

TV

팩트체크

알고리즘 안내

정정보도 모음

디지털타임스

디지털타임스

구독

디지털타임스 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다.
보러가기
닫기

디지털타임스 언론사 구독 해지

--- hit 2 ---
source: https://n.news.naver.com/mnews/article/029/0002927209
김나인 기자(silkni@dt.co.kr)

--- hit 3 ---
source: https://n.news.naver.com/mnews/article/008/0005136824
"게임 만들 사람 줄어도 괜찮아" AI가 책임진다

본문 바로가기

NAVER

뉴스

엔터

스포츠

날씨

프리미엄

검색

언론사별

정치

경제

사회

생활/문화

IT/과학

세계

랭킹

신문보기

오피니언

TV

팩트체크

알고리즘 안내

정정보도 모음

머니투데이

머니투데이

구독

머니투데이 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다.
보러가기
닫기

머니투데이 언론사 구독 해지되었습니다.
닫기

"

--- hit 4 ---
source: https://n.news.naver.com/mnews/article/092/0002358620
AI에 진심인 MS, 1년간 데이터센터에 118兆 붓는다

본문 바로가기

NAVER

뉴스

엔터

스포츠

날

### 실습 프로젝트

[RAG 파이프라인 구축하기] 이제 배운 내용을 바탕으로 실제 RAG 파이프라인을 직접 구축해보겠습니다.

**목표**: 뉴스 기사를 로드하고, 청크로 분할하여 벡터 저장소에 저장한 후, 질문에 답변하는 RAG 시스템 구축

**단계별 가이드**:
1. 데이터 준비 → 텍스트 처리 → 임베딩 → 검색 → 평가
2. 각 과제마다 구현해야 할 함수와 결과물 정의
3. 단위 테스트를 통한 검증

In [5]:
# 1단계: 데이터 준비 - 웹문서 검색을 위해 관련 URL 가져오기
web_urls = [
    "https://n.news.naver.com/mnews/article/029/0002927209",
    "https://n.news.naver.com/mnews/article/092/0002358620",
    "https://n.news.naver.com/mnews/article/008/0005136824",
]

In [6]:
# 2단계: WebBaseLoader를 사용해 텍스트 로드
"""
힌트:
- WebBaseLoader를 사용하여 web_urls의 문서들을 로드하세요
- loader.load()를 호출하여 Document 객체 리스트를 얻습니다
- 로드된 문서의 개수를 출력하여 확인하세요

기대 출력:
- 로드된 문서 개수: 3개 (URL 개수와 동일)
"""
pass

In [7]:
# 3단계: CharacterTextSplitter로 문서 분할
"""
힌트:
- CharacterTextSplitter를 import하세요
- chunk_size=500, chunk_overlap=100으로 설정하세요
- separator="\n\n"로 문단 단위 분할
- split_documents() 메서드를 사용하여 문서를 분할하세요
- 분할된 청크의 개수를 출력하세요

기대 출력:
- 분할된 청크 개수: 약 20-40개 (문서 내용에 따라 다를 수 있음)
"""
pass

In [8]:
# 4단계: 임베딩 및 벡터 저장소 구현
"""
힌트:
- OpenAIEmbeddings를 사용하여 임베딩 모델을 생성하세요
- Chroma 벡터 저장소를 생성하세요
- add_documents()를 사용하여 분할된 문서를 벡터 저장소에 추가하세요
- 저장된 문서 개수를 확인하세요

기대 출력:
- 임베딩 모델 생성 완료
- 벡터 저장소에 저장된 문서 개수: (3단계의 청크 개수와 동일)
"""
pass

In [9]:
# 5단계: RAG 기반 QA 체인 구현
"""
힌트:
- ChatOpenAI로 LLM 모델을 생성하세요 (model="gpt-4.1-nano")
- ChatPromptTemplate으로 프롬프트 템플릿을 만드세요
- 벡터 저장소에서 as_retriever()로 검색기를 생성하세요
- RunnableParallel 사용하여 RAG 체인을 구성하세요

기대 출력:
- RAG 체인 생성 완료
"""
pass

In [10]:
# 6단계: QA 체인으로 질문 응답
"""
힌트:
- 뉴스 기사와 관련된 질문을 준비하세요
- rag_chain.invoke({"question": 질문})으로 답변을 얻으세요
- response['answer']로 답변 내용을 확인하세요
- response['context']로 검색된 문서를 확인하세요

기대 출력:
- 질문에 대한 답변이 출력됩니다
- 검색된 관련 문서들의 내용이 포함됩니다

예시 질문:
- "기사의 주요 내용을 요약해주세요"
- "주요 인물은 누구인가요?"
- "어떤 사건에 대한 기사인가요?"
"""
pass
